#Watch a directory and plot fits files as they appear

The Wathcher example class can be found online (either at watchdog docs or from Cho https://www.michaelcho.me/article/using-pythons-watchdog-to-monitor-changes-to-a-directory).

In this one we count fits files and render every Nth one  to the notebook.

In [ ]:
import sys
import os
from watch import FitsWatcher 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
        

This is the the Main loop , here you must enter the directory to monitor and the number of files to skip over before rendering one. The output will be in the next cell after this. 

In [ ]:
    d = "tmp"
    skip = 2
    w = FitsWatcher(d,skip)
    w.run()
